In [1]:
from playsound import playsound
playsound('C:/Users/joeca/Desktop/Finance/FOREX/Hercules_Prototype_Mark_II/MP3s/startup.mp3')
from datetime import datetime as dt
from collections import defaultdict
import time
import numpy as np
import pandas as pd
import math
from scipy import stats
from scipy.signal import argrelextrema
from scipy.signal import savgol_filter
import statsmodels.api as sm
from sympy import *
import pytz
import itertools
import matplotlib.pyplot as plt
from itertools import combinations
from itertools import product
from itertools import groupby
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import MetaTrader5 as mt5

In [2]:
if not mt5.initialize(path="C:/JoeC/MetaTrader 5/terminal64.exe",login=1045610, server="TradersWay-Demo",password="r5AH8znU"):
    print("initialize() failed, error code =",mt5.last_error())
    quit()

In [3]:
def levelgrab(pair):
    
    levslist = []
    
    for x in pair:

        Srates = mt5.copy_rates_from_pos(x, mt5.TIMEFRAME_H8, 0, 600)
        S = Srates
        SCM = np.average(abs(S['close'] - S['open']))
        y = S['close']
        LVL = np.concatenate((y[argrelextrema(y, np.greater)[0]],y[argrelextrema(y, np.less)[0]]))
        B = list(combinations(LVL, 2))
        Ba = np.array([x for x, y in B])
        B = np.array([abs(x - y) for x, y in B])
        Bb = np.array([np.where(x <= SCM, 1, 0) for x in B])
        T = list(zip(Ba,Bb))
        table = pd.DataFrame(T, columns=['a','b']).groupby('a').sum()
        table['max'] = table.b[(table.b.shift(1) < table.b) & (table.b.shift(-1) < table.b)]
        table = table.sort_values(['a'], ascending=False)
        table = table.fillna(0)
        table = table[table['max'] > np.average(table['b'])]
        table = table.assign(level=table.index)
        levels = list(table['level'])
        
        S = pd.DataFrame(Srates)
        S['time']=pd.to_datetime(S['time'], unit='s')
        S = S.set_index('time')
        S = S.assign(MOVE=(S['close'] - S['open']))
        SCM = np.average(abs(S['MOVE']))
        CNDL = (S.iloc[:, 3]) - (S.iloc[:, 0])
        S['CNDL'] = np.where(CNDL > 0, 1, -1)
        S = S.assign(INFL=S['CNDL'].ne(S['CNDL'].shift()))
        S = S[S.INFL != 0]
        S['INFL'] = S['CNDL']*-1
        S['LVL'] = np.where(S.INFL == 1, S.high, S.low)

        A = np.array(S.LVL)
        B = list(combinations(A, 2))
        Ba = np.array([x for x, y in B])
        B = np.array([abs(x - y) for x, y in B])
        Bb = np.array([np.where(x <= SCM, 1, 0) for x in B])
        T = list(zip(Ba,Bb))
        table = pd.DataFrame(T, columns=['a','b']).groupby('a').sum()
        table['max'] = table.b[(table.b.shift(1) < table.b) & (table.b.shift(-1) < table.b)]
        table = table.sort_values(['a'], ascending=False)
        table = table.rename(columns={'b': 'Pings'})
        table = table.fillna(0)
        table = table[table['max'] != 0]
        pingsavg = np.average(table['Pings'])
        pingsstd = np.std(table['Pings'])
        #table = table[table['max'] > (pingsavg+pingsstd)]
        table = table[table['max'] > pingsavg]
        table = table.assign(level=table.index)
        levs = tuple(table['level'])
        
        print(levs)
        
        #levslist.append(levs)
        
    #print(levslist)

    #return levslist


In [4]:
def levelgrabbeta(pair):
    
    levslist = []
    
    for x in pair:
        Srates = mt5.copy_rates_from_pos(x, mt5.TIMEFRAME_H8, 0, 600)
        S = Srates
        SCM = np.average(abs(S['close'] - S['open']))
        y = S['close']
        LVL = np.concatenate((y[argrelextrema(y, np.greater)[0]],y[argrelextrema(y, np.less)[0]]))
        B = list(combinations(LVL, 2))
        Ba = np.array([x for x, y in B])
        B = np.array([abs(x - y) for x, y in B])
        Bb = np.array([np.where(x <= SCM, 1, 0) for x in B])
        T = [(Ba[i], Bb[i]) for i in range(0, len(Ba))] 
        Toutput = defaultdict(int) 
        for k, v in T: 
            Toutput[k] += v 
        table = list(Toutput.items())
        tablevs = [x for x, y in table]
        tabpings = [y for x, y in table]
        tabmax = argrelextrema(np.array(tabpings), np.greater)
        tabmax2 = list(tabmax[0])
        levs = np.array(tablevs)[tabmax2]
        levs = list(levs)
        levs.sort(reverse=True)
        levs = list(levs)
        
        mindist = min([abs(j-i) for i, j in zip(levs[:-1], levs[1:])])
        
        while mindist < SCM:
            i = 0
            h = -1
            j = 1
            for c in range(0, len(levs)):
                if j == len(levs):
                    j = -1
                if abs(levs[i]-levs[h]) < SCM:
                    levs[i] = 0
                elif abs(levs[i]-levs[j]) < SCM:
                    levs[i] = levs[i] - abs(levs[i]-levs[j])/2
                i += 1
                h += 1
                j += 1
            levs = [v for v in levs if v != 0]
            mindist = min([abs(a-b) for a, b in zip(levs[:-1], levs[1:])])
            
        levs = tuple(levs)

        levslist.append(levs)
        
        

In [5]:
print("Initializing...")

#pair = ['AUDCAD','AUDCHF','AUDJPY','AUDNZD','AUDUSD','CADCHF','CADJPY','CHFJPY','EURAUD','EURCAD','EURCHF','EURGBP','EURJPY','EURNZD','EURUSD','GBPAUD','GBPCAD','GBPCHF','GBPJPY','GBPUSD','NZDCAD','NZDCHF','NZDJPY','NZDUSD','USDCAD','USDCHF','USDJPY']
pair = ['AUDCAD']
#levelgrab(pair)
levelgrabbeta(pair)

Initializing...
SCM:     0.0021014166666666646
mindist:  9.99999999995449e-06
first levels       [0.93378, 0.93374, 0.93165, 0.91478, 0.91043, 0.90896, 0.90891, 0.90888, 0.90809, 0.90808, 0.90775, 0.90772, 0.9075, 0.90719, 0.90714, 0.90686, 0.90683, 0.90681, 0.90668, 0.90666, 0.90642, 0.90603, 0.90572, 0.90178, 0.90159, 0.90149, 0.90128, 0.9012, 0.90119, 0.90102, 0.90095, 0.9008, 0.90052, 0.90034, 0.90018, 0.89975, 0.8995, 0.89944, 0.89897, 0.89874, 0.89871, 0.89862, 0.8984, 0.89832, 0.89813, 0.89799, 0.89765, 0.89762, 0.89749, 0.89647, 0.8946, 0.89371, 0.89303, 0.89295, 0.89204, 0.89201, 0.89159, 0.89121, 0.88967, 0.88956, 0.88925, 0.88727, 0.88709, 0.88667, 0.88619, 0.8859, 0.88556, 0.88551, 0.88524, 0.87656, 0.87543, 0.87209, 0.86196, 0.85556, 0.83126]
SCM:     0.0021014166666666646
mindist:  0.00014999999999987246
[0.93376, 0.93165, 0.91478, 0.9096949999999999, 0.908895, 0.908085, 0.907735, 0.9073450000000001, 0.907, 0.90682, 0.9066700000000001, 0.9062250000000001, 0.90572, 0.90168